In [1]:
from sac import Actor, SoftQNetwork, train_sac

observation_shape = [10,]
action_shape = [3,]
action_space_high =[1,1,1]
action_space_low  =[-1,-1,-1]
actor = Actor(observation_shape, action_shape, action_space_high, action_space_low)

import copy
qf1 = SoftQNetwork(observation_shape, action_shape)
qf2 = SoftQNetwork(observation_shape, action_shape)
qf1_target = copy.deepcopy(qf1)
qf2_target = copy.deepcopy(qf2)

actor


2024-10-30 17:22:36.391317: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-30 17:22:36.404529: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-30 17:22:36.420855: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-30 17:22:36.425702: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-30 17:22:36.437538: I tensorflow/core/platform/cpu_feature_guar

Actor(
  (fc1): Linear(in_features=10, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (fc_mean): Linear(in_features=256, out_features=3, bias=True)
  (fc_logstd): Linear(in_features=256, out_features=3, bias=True)
)

In [ ]:
import numpy as np 
import gymnasium as gym
class CustomEnv():
    def __init__(self):
        action_dim = 3
        obs_dim = 10
        self.single_action_space  = gym.spaces.Box(-np.inf, np.inf, shape=(action_dim,))
        self.single_observation_space  = gym.spaces.Box(-np.inf, np.inf, shape=(obs_dim,), dtype=np.float32)
        print(self.single_action_space)
        print(self.single_observation_space)
        
    def step(self, action):
        state = self.obs()
        reward = 1 
        done = False if np.random.random() < 0.95 else True 
        self.info['episode']['l'] += 1 
        self.info['episode']['r'] += reward
        self.done = done
        return [state], [reward], [done], [done], [self.info] 
    
    def reset(self, seed=0):
        self.done = False 
        state= self.obs()
        self.info = {
            'episode':{
                'r': 0,
                'l': 0,
            }
        }
        return state, self.info 
    
    def obs(self):
        if self.done:
            obs, _ = self.reset()
            return obs 
        else:
            return np.random.random(10,)

envs = CustomEnv()
train_sac(envs, actor, qf1, qf2, qf1_target, qf2_target, run_name='test', 
          device='cuda',
          total_timesteps=100000,
          learning_starts=1000,
)

Box(-inf, inf, (3,), float32)
Box(-inf, inf, (10,), float32)
set:device = cuda
set:total_timesteps = 100000
set:learning_starts = 1000


  1%|          | 974/100000 [00:00<00:10, 9376.97it/s, return=22]/data1/bumjin/PybulletDroneRL/sac.py:178: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  actions, _, _ = actor.get_action(torch.Tensor(obs).to(device))
  2%|▏         | 1826/100000 [00:12<00:10, 9376.97it/s, return=24]